In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install EMD-signal

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.7 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import time
import scipy
import scipy.io
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.utils import data
from torch.autograd import Variable
import pandas as pd
from PyEMD import EEMD

In [3]:
import sys    
sys_path = '/content/drive/MyDrive/EMG-classification'
sys.path.append(sys_path)
from util import *
from dataloader import *

In [8]:
# DataLoader

train_record_data = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0501.npy')
train_label = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0501.npy')

train_signal = train_record_data.flatten()
print("train data:{}\n".format(train_record_data.shape))

train data:(5746, 1200)



In [26]:
class Netone(nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers, middle_feature):
        super(Netone, self).__init__()
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = False
        self.middle_feature = middle_feature

        self.embedding = nn.Sequential(
            nn.Conv2d(1, 2, kernel_size=(3, 1), padding=(1, 0), stride=1),
            nn.ReLU(),
            nn.Conv2d(2, 4, kernel_size=(3, 1), padding=(1, 0), stride=1),
            nn.ReLU())

        # self.embedding_dim = 1281
        self.embedding_dim = 129*4
        # h0 = 256 num_layers = 1

        self.BiLSTM = nn.LSTM(input_size=self.embedding_dim,
                              hidden_size=self.hidden_size,
                              num_layers=self.num_layers,
                              dropout=0.2,
                              bidirectional=self.bidirectional)

        self.MLP = nn.Sequential(
            nn.Linear(self.hidden_size, self.middle_feature),
            nn.ReLU(),
            nn.Linear(self.middle_feature, num_classes)
        )  # remove the mlp and generate the irritation state at each time step

    def forward(self, datas):
        batch, f, t = datas.shape
        # print('input:', batch, f, t)# overall patient data = [1, 129, 54]
        # datas = datas.view(batch, 1, t, f)  # [N, C, H, W]
        datas = datas.reshape(batch, 1, t, f)
        embedded_datas = self.embedding(datas)
        # print('after conv:',embedded_datas.size()) # [1, 4, 56, 129]
        # embedded_datas = embedded_datas.permute(3, 0, 1, 2).view(f, batch, -1)
        embedded_datas = embedded_datas.permute(2, 0, 1, 3).reshape(t, batch, -1)
        # print('before lstm:',embedded_datas.size())
        out, _ = self.BiLSTM(embedded_datas)
        # print('after lstm:',out.size()) # [56, 1, 256]
        # out = out[-1, :, :]  # 20,512
        out = self.MLP(out.squeeze())
        # print('output shape:', out.shape)
        return out

In [ ]:
eps = 1
input_size = 129
#set hidden size
hidden_size = 256
n_class = 9
learning_rate = 0.01
max_epochs = 20
middle_features = 128
model = Netone(num_classes=n_class, hidden_size=hidden_size, num_layers=1, middle_feature=middle_features) 
#num_classes, hidden_size, num_layers, bidirectional, middle_feature
#model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train_acc = []
val_acc = []

for epoch in range(max_epochs):
    correct_all = 0
    total_loss = 0
    train_length = 0
    predicted_train_csv = []
    real_train_csv = []

    for batch_idx in range(0,train_record_data.shape[0],eps):
        train_eIMFs = train_signal[1200*batch_idx:1200*(batch_idx+eps)+1]
        train_raw = spectro(train_record_data[batch_idx:batch_idx+eps], train_eIMFs, 100, 256)
        train_raw = torch.from_numpy(train_raw)
        label_raw = train_label[batch_idx:batch_idx+eps]
        # print("train_raw shape:{}\n label_raw shape:{}".format(train_raw.shape, label_raw.shape)) # (1, 129, 56)

        label = np.repeat(label_raw, train_raw.shape[-1])
        label = torch.from_numpy(label)
        # train_set = train_raw.permute(0, 2, 1)
        # train_set = train_set.reshape(train_set.shape[0]*train_set.shape[1], -1)
        # print("train_set shape:{}\nlabel shape:{}\n".format(train_set.shape, label.shape)) # [310284, 129]

        # batch_x = torch.unsqueeze(train_set, dim=0) # [1, 263736, 129]
        batch_x = train_raw
        batch_x = Variable(batch_x.float())  # .cuda()
        batch_y = label.long()

        optimizer.zero_grad()
        output = model(batch_x)  # ,hidden = model(batch_x,None)

        # toc = time.perf_counter()
        # print(f"test epochs time in {toc - tic:0.4f} seconds")

        _, predicted_train = torch.max(output.data, 1)
        assert(len(predicted_train)==len(batch_y))
        correct_all += sum(batch_y.eq(predicted_train))
        train_length += len(batch_y)

        predicted_train_csv.append(predicted_train.cpu().detach().numpy())
        real_train_csv.append(batch_y.cpu().detach().numpy())

        loss = criterion(output, batch_y.squeeze())#.float().reshape(-1, 1)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print('\n************************************************************************************************')
    print("Epoch {}/{}:, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch+1,
        max_epochs,
        float(total_loss / train_length),
        float(optimizer.param_groups[0]['lr'])))
    print('train_acc:{}\n'.format(correct_all/train_length))